<a href="https://colab.research.google.com/github/mayanku1111/Fake_news-classification-model-using-lstm/blob/main/Fake_newsClassification_lstm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [ ]:
df=pd.read_csv('train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.15.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mayankkumar/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = df.dropna()

In [ ]:
df.shape


(18285, 5)

In [ ]:
X=df.drop('label',axis=1)
y=df['label']

In [ ]:
X.shape

(18285, 4)

In [ ]:
y.shape

(18285,)

In [ ]:
voc_size=5000

In [ ]:
messages=X.copy()

In [ ]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
messages.reset_index(inplace=True)

In [2]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [3]:
corpus

In [4]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

# embeddings now

In [ ]:
sent_length=25
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  121 4533 4130]
 [   0    0    0 ... 1517 1896 2568]
 [   0    0    0 ... 4364 2476 4882]
 ...
 [   0    0    0 ... 2836 4948 3522]
 [   0    0    0 ... 1655 1681 3083]
 [   0    0    0 ... 3153 3673 4840]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 4045, 1111, 1500, 1713, 1911,  204, 4552,
        121, 4533, 4130], dtype=int32)

In [ ]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(18285, (18285,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((18285, 25), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 3s 13ms/step - loss: 0.3307 - accuracy: 0.8460 - val_loss: 0.1987 - val_accuracy: 0.9181
Epoch 2/10
192/192 [==============================] - 2s 13ms/step - loss: 0.1399 - accuracy: 0.9458 - val_loss: 0.1973 - val_accuracy: 0.9233
Epoch 3/10
192/192 [==============================] - 2s 13ms/step - loss: 0.0964 - accuracy: 0.9647 - val_loss: 0.2394 - val_accuracy: 0.9203
Epoch 4/10
192/192 [==============================] - 2s 13ms/step - loss: 0.0682 - accuracy: 0.9748 - val_loss: 0.2704 - val_accuracy: 0.9176
Epoch 5/10
192/192 [==============================] - 2s 13ms/step - loss: 0.0457 - accuracy: 0.9854 - val_loss: 0.3223 - val_accuracy: 0.9115
Epoch 6/10
192/192 [==============================] - 2s 13ms/step - loss: 0.0266 - accuracy: 0.9915 - val_loss: 0.3577 - val_accuracy: 0.9142
Epoch 7/10
192/192 [==============================] - 2s 13ms/step - loss: 0.0160 - accuracy: 0.9950 - val_loss: 0.4166 - val_accuracy: 0.9142

# quantization and dropout

In [ ]:
from tensorflow.keras.layers import Dropout


embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.4))
model.add(LSTM(100))
model.add(Dropout(0.4))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 25, 40)            200000    
                                                                 
 dropout_4 (Dropout)         (None, 25, 40)            0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 3s 14ms/step - loss: 0.3518 - accuracy: 0.8263 - val_loss: 0.2016 - val_accuracy: 0.9168
Epoch 2/10
192/192 [==============================] - 3s 13ms/step - loss: 0.1537 - accuracy: 0.9399 - val_loss: 0.1982 - val_accuracy: 0.9215
Epoch 3/10
192/192 [==============================] - 3s 13ms/step - loss: 0.1187 - accuracy: 0.9568 - val_loss: 0.2179 - val_accuracy: 0.9211
Epoch 4/10
192/192 [==============================] - 2s 13ms/step - loss: 0.0945 - accuracy: 0.9649 - val_loss: 0.2303 - val_accuracy: 0.9173
Epoch 5/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0749 - accuracy: 0.9730 - val_loss: 0.2363 - val_accuracy: 0.9175
Epoch 6/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0605 - accuracy: 0.9781 - val_loss: 0.2962 - val_accuracy: 0.9198
Epoch 7/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0475 - accuracy: 0.9827 - val_loss: 0.3137 - val_accuracy: 0.9153

In [ ]:
model.evaluate(X_test,y_test)

189/189 [==============================] - 0s 2ms/step - loss: 0.4256 - accuracy: 0.9054


[0.4255804717540741, 0.9053852558135986]

In [ ]:
model.save("./saved_model")

INFO:tensorflow:Assets written to: ./saved_model/assets


INFO:tensorflow:Assets written to: ./saved_model/assets


In [5]:
import tensorflow as tf

def representative_dataset_gen():
    for _ in range(num_calibration_steps):
        yield [input]

converter = tf.lite.TFLiteConverter.from_saved_model("./saved_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
tflite_quant_model = converter.convert()

In [ ]:
with open("tflite_quant_model.tflite", "wb") as f:
    f.write(tflite_quant_model)